In [1]:
import json
from urllib.parse import urlencode
from urllib.request import urlretrieve
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import requests
from lxml import html

import pickle


In [37]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}

    useragent = "\"Mozilla/5.0 (X11; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(pd.DataFrame(d, columns=h))

Start by exploring combine data

In [5]:
params = {'LeagueID':'00',
          'Season':'2017-18',
          'IsOnlyCurrentSeason':'0'
         }
data = get_nba_data(endpt="commonallplayers",params=params)
data['PLAYER_NAME']=pd.Series(data['DISPLAY_FIRST_LAST'],dtype=str)+"("+pd.Series(data.FROM_YEAR,dtype=str)+")"
data = data.set_index('PLAYER_NAME')
data['PLAYER_ID']=data['PERSON_ID']
Player_ID_Dict = pd.Series(data['PLAYER_ID'],dtype=int)
Player_ID_Dict
Player_ID_Dict = Player_ID_Dict.loc[~Player_ID_Dict.index.duplicated(keep='first')]

pickle.dump(Player_ID_Dict, open('NBAPlayerDict.pkl', 'wb'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
year = "2012-13"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinedrillresults"
df1 = get_nba_data(endpt=endpt,params=params)

In [ ]:
endpt = "draftcombineplayeranthro"
df2 = get_nba_data(endpt=endpt,params=params)

In [ ]:
year = "2013-14"
params = {"LeagueID":"00",
          "SeasonYear":year}
endpt = "draftcombinenonstationaryshooting"
df3 = get_nba_data(endpt=endpt,params=params)

In [ ]:
df = pd.merge(df1,df2,on="PLAYER_ID")

Not looking like we'll have the shooting data very constitently. Lets merge two we have consitently and compile it over several years

In [6]:
dfCombineFull = pd.DataFrame(columns=df.columns)
for i in range(10,13):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2,on="PLAYER_ID")

    if(i>13):
        dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
        dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfCombineFull = pd.concat([dfCombineFull,dfMerged])

NameError: name 'df' is not defined

In [7]:
#dfCombineFull = dfCombineFull.set_index('PLAYER_ID')
dfCombineFull[dfCombineFull.LAST_NAME_x=="Davis"]

NameError: name 'dfCombineFull' is not defined

In [127]:
endpt = 'playerdashptshotdefend'
params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
dfDefense = get_nba_data(endpt=endpt,params=params)
dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]


IndexError: list index out of range

In [11]:
dfDefense = dfDefense[dfDefense.DEFENSE_CATEGORY=="Less Than 6 Ft"]
dfDefense = dfDefense.set_index("CLOSE_DEF_PERSON_ID")

NameError: name 'dfDefense' is not defined

In [12]:
dfDefense_PCT_Change = pd.DataFrame(dfDefense.PCT_PLUSMINUS)
dfDefense_PCT_Change

NameError: name 'dfDefense' is not defined

In [13]:
dfCombineFull = pd.merge(dfCombineFull,dfDefense_PCT_Change,left_index=True,right_index=True)

NameError: name 'dfDefense_PCT_Change' is not defined

In [14]:
dfCombineFull.groupby('WINGSPAN')['PCT_PLUSMINUS'].mean()

KeyError: 'Column not found: PCT_PLUSMINUS'

In [15]:
plt.plot(dfCombineFull.groupby('WINGSPAN')['PCT_PLUSMINUS'].mean())

KeyError: 'Column not found: PCT_PLUSMINUS'

In [16]:
bins = list(range(70,98,2))
dfCombineFull['Wingspan_binned']=pd.cut(dfCombineFull['WINGSPAN'],bins)

In [17]:
dfCombineFull['POSITION_SIMPLE']=np.NaN
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="C"]="Big"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="C-PF"]="Big"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF-C"]="Big"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF"]="Forward"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF-PF"]="Forward"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PF-SF"]="Forward"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF"]="Wing"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SF-SG"]="Wing"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG-SF"]="Wing"

dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PG-SG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="SG-PG"]="Guard"
dfCombineFull['POSITION_SIMPLE'][dfCombineFull.POSITION_y=="PG"]="Guard"




dfCombineFull.groupby(['POSITION_SIMPLE','Wingspan_binned'])['PCT_PLUSMINUS'].mean()


/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dhoskins/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/dh

KeyError: 'Column not found: PCT_PLUSMINUS'

In [ ]:
plt.scatter(dfCombineFull['THREE_QUARTER_SPRINT'],dfCombineFull['PCT_PLUSMINUS'])

In [ ]:
dfCombineFull.columns

Tomorrow we should work on a simple linear regression from this.
We should also start adding in college status
Need to do data from multiple years/figure out what we want to do regarding that
    averages over full career
    best year
    three years out?
    three year average?

In [8]:
year = 16
year="20"+format(i,'02d')+"-"+format(i+1,'02d')
params.update({"SeasonYear":year})
df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
if(i>13):
    dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
    
dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)


dfCombineFull = pd.concat([dfCombineFull,dfMerged])
dfMerged

NameError: name 'i' is not defined

In [9]:
Player_ID_Dict = Player_ID_Dict.loc[~Player_ID_Dict.index.duplicated(keep='first')]

NameError: name 'Player_ID_Dict' is not defined

In [10]:
Player_ID_Dict['Anthony Davis(20)']

NameError: name 'Player_ID_Dict' is not defined

Going to start pulling college stats from bball ref but might get my IP banned which is less than ideal.

In [11]:
nameList = []
for player in dfCombineFull.index:
    nameStr = 'https://www.sports-reference.com/cbb/players/'
    
    nameStr += str(dfCombineFull.FIRST_NAME_x[player]).lower() +"-" + \
                str(dfCombineFull.LAST_NAME_x[player]).lower() + "-1.html"
    
    nameList.append(nameStr)

NameError: name 'dfCombineFull' is not defined

In [12]:
testCase = 'https://www.sports-reference.com/cbb/players/deandre-ayton-1.html'

In [13]:
data_url = 'https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html'
user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

headers = {'User-Agent':user_agent}
page = requests.get(data_url,headers=headers)
tree = html.fromstring(page.content)

In [14]:

x = tree.xpath('//div[@id="all_players_per_poss"]')
n = len(str(x[0].getchildren()[-1]))

tree2 = html.fromstring(str(x[0].getchildren()[-1])[6:(n-3)])
tree2.xpath('//table/tfoot/tr/td/text()')

['Virginia',
 '136',
 '109',
 '4157',
 '9.6',
 '22.3',
 '.430',
 '6.6',
 '14.2',
 '.466',
 '2.9',
 '8.1',
 '.365',
 '6.7',
 '7.7',
 '.876',
 '9.0',
 '5.3',
 '1.9',
 '0.5',
 '3.3',
 '4.0',
 '28.8',
 '114.9',
 '94.0']

In [15]:
x = tree.xpath('//div[@id="all_leaderboard"]')
n = len(str(x[0].getchildren()[-1]))

tree2 = html.fromstring(str(x[0].getchildren()[-1])[6:(n-3)])
tree2.xpath('//table/tr/td/a/text')

[]

In [16]:
x = soup.find_all('td')

NameError: name 'soup' is not defined

In [17]:
//*[@id="all_leaderboard"]

SyntaxError: invalid syntax (<ipython-input-17-fc43eab94dfd>, line 1)

In [18]:
nameList

[]

In [19]:
#dfCombineFull = pd.DataFrame(columns=df.columns)
i = 17
year="20"+format(i,'02d')+"-"+format(i+1,'02d')
params.update({"SeasonYear":year})
df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)


dfMerged = pd.merge(df1,df2)

if(i>13):
    dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME'] + "(" + year[0:4] + ")"
    dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    
dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

dfMerged = dfMerged[dfMerged['MAX_VERTICAL_LEAP']!=np.nan]\
                    .set_index('PLAYER_ID')\
                    .drop(['FIRST_NAME','LAST_NAME'],axis=1)
#.set_index('PLAYER_ID')
#dfCombineFull = pd.concat([dfCombineFull,dfMerged])
dfMerged

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [20]:
nameList = []
yearList = []
for name in dfMerged.PLAYER_NAME:
    nameStr = 'https://www.sports-reference.com/cbb/players/'
    first = re.sub('[^a-zA-Z]+', '',name.split()[0].lower())
    last = "".join(name.split()[1:])[0:-6].lower().strip(".")
    year = name.split()[1][-5:-1]
    yearList.append(year)
    nameStr += first + "-" + last + "-"
    nameList.append(nameStr)
    


NameError: name 'dfMerged' is not defined

In [21]:
nameList

[]

In [22]:
rowDf = pd.DataFrame(columns=row.columns)
for indx,name in enumerate(nameList):
    stopCondition = False
    year = yearList[indx]
    i = 1

    while not stopCondition:
        data_url = name + str(i) + ".html"
        print(data_url)
        player_id = dfMerged.index[indx]
        player_name = dfMerged.PLAYER_NAME[indx]
        user_agent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

        headers = {'User-Agent':user_agent}
        page = requests.get(data_url,headers=headers)
        tree = html.fromstring(page.content)

        if '404' not in tree.xpath('//title')[0].text:
            finalYear = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]

            if (finalYear != year):
                i+=1
                continue
            else:
                stopCondition = True

            treeStr = str(tree.xpath('//div[@id="all_players_per_poss"]')[0]\
                            .getchildren()[-1])[8:-4]

            tree2 = html.fromstring(treeStr)
            valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
            conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
            colNamesHTML = tree2.xpath('//table/thead/tr/th')
            colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
            values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]

            row = pd.DataFrame(pd.Series(values,index=colNames)\
                    .dropna()).transpose().set_index("PLAYER_ID")

        else:
            stopCondition=True
            row = None
    rowDf = pd.concat([rowDf,row])
    print(indx)
    
#Fixing juniors is going to require a functional approach to this. 

NameError: name 'row' is not defined

In [23]:
len(dfMerged)

NameError: name 'dfMerged' is not defined

Need to make that into a function soon. Fuck it why not now.

In [38]:
def SportsRefCBBRowMaker(player_url,table_name,player_name,correct_year):
    player_id = Player_ID_Dict[player_name]
    
    page = requests.get(player_url,headers=headers)
    tree = html.fromstring(page.content)
    
    if '404' in tree.xpath('//title')[0].text:
        return('404')
    
    else:
        year = tree.xpath('//table/tbody/tr')[-1].items()[0][-1][-4:]
        if(year!=correct_year):
            return(year)
        path = '//div[@id="'+table_name+'"]'
        treeStr = str(tree.xpath(path)[0].getchildren()[-1])[8:-4]
        #This is tricky and comments might be nice here. 

        tree2 = html.fromstring(treeStr)
        
        valuesHTML = tree2.xpath('//table/tfoot/tr')[-1].getchildren()[1:]
        ## This gives career in particular 
        conf = tree2.xpath('//table/tbody/tr/td')[1].getchildren()[0].text
        colNamesHTML = tree2.xpath('//table/thead/tr/th')
        colNames = ["PLAYER_ID","PLAYER_NAME"]+["CONF"] + [col.text for col in colNamesHTML][1:]
        values = [player_id,player_name] + [conf] + [val.text for val in valuesHTML]

        row = pd.DataFrame(pd.Series(values,index=colNames)\
                .dropna()).transpose().set_index("PLAYER_ID")
 #       print(player_url)
        return(row)
    

In [39]:
def sportsRefCBBTableMaker(player_df, table_name):
    base_url = 'https://www.sports-reference.com/cbb/players/'
    columns_df_url = "https://www.sports-reference.com/cbb/players/malcolm-brogdon-1.html"
    columns_df = SportsRefCBBRowMaker(columns_df_url,table_name
                                      ,"Malcolm Brogdon(2016)",'2016')
    CBB_df = pd.DataFrame(columns=columns_df.columns)
    for index,row in dfMerged.iterrows():
        player_name = row['PLAYER_NAME']
        first = re.sub('[^a-zA-Z]+', '',player_name.split()[0].lower())
        last = "".join(player_name.split()[1:])[0:-6].lower().strip(".")
        year = player_name.split()[1][-5:-1]
        name_url = base_url + first + "-" + last + "-"

        i = 1
        stopping_condition=False
        while(stopping_condition==False):
            data_url = name_url + str(i) + ".html"
#            print(data_url)
            row = SportsRefCBBRowMaker(data_url,table_name,player_name,year)
            if isinstance(row, pd.DataFrame):
                stopping_condition=True
                continue
            if(row=='404'):
                stopping_condition=True
                row = None
                if(last[-2:]=="jr"):
                    last = last[:-2]
                    name_url = base_url + first + "-" + last + "-"
                    i=1
                    stopping_condition = False 
            i+=1
        CBB_df = pd.concat([CBB_df,row])
    return(CBB_df)

In [40]:
sportsRefCBBTableMaker(dfMerged, "all_players_per_poss")

NameError: name 'dfMerged' is not defined

In [41]:
dfCombineFull = pd.DataFrame(columns=df.columns)
for i in range(10,13):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2,on="PLAYER_ID")

    if(i>13):
        dfMerged = pd.merge(df1,df2,on="TEMP_PLAYER_ID")
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME_x'] + "(" + year[0:4] + ")"
        dfMerged['PLAYER_ID']=dfMerged['PLAYER_NAME'].map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfCombineFull = pd.concat([dfCombineFull,dfMerged])

NameError: name 'df' is not defined

In [42]:
full_combine_df = pd.DataFrame(columns = combine_columns)
combine_columns = ['PLAYER_NAME','DRAFT_YEAR','POSITION','STANDING_VERTICAL_LEAP',
                   'MAX_VERTICAL_LEAP','LANE_AGILITY_TIME',
                   'MODIFIED_LANE_AGILITY_TIME','THREE_QUARTER_SPRINT',
                   'BENCH_PRESS','HEIGHT_WO_SHOES','WEIGHT','WINGSPAN',
                   'STANDING_REACH','BODY_FAT_PCT','HAND_LENGTH','HAND_WIDTH'                  
                  ]
for i in range(0,19):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({"SeasonYear":year})
    df1 = get_nba_data(endpt="draftcombinedrillresults",params=params)
    df2 = get_nba_data(endpt="draftcombineplayeranthro",params=params)
    dfMerged = pd.merge(df1,df2)
    dfMerged['PLAYER_ID']=pd.Series(dfMerged.PLAYER_ID,dtype=str)
    if(i>13):
        dfMerged['PLAYER_NAME'] = dfMerged['PLAYER_NAME']
        dfMerged['PLAYER_ID']=(dfMerged['PLAYER_NAME']+"("+year[0:4]+")").map(Player_ID_Dict)
        dfMerged['PLAYER_ID']=pd.Series(dfMerged['PLAYER_ID'],dtype=str).str.split(".").str.get(0)

    dfMerged['DRAFT_YEAR'] = int(year[0:4])    
    dfMerged = dfMerged[dfMerged['PLAYER_ID']!='nan']

    dfMerged = dfMerged[dfMerged['MAX_VERTICAL_LEAP']!=np.nan]\
                        .set_index('PLAYER_ID')[combine_columns]
        
    full_combine_df = pd.concat([full_combine_df,dfMerged])
full_combine_df.fillna(value=np.nan, inplace=True)
full_combine_df

NameError: name 'combine_columns' is not defined

In [43]:
full_combine_df.groupby('DRAFT_YEAR').count()

NameError: name 'full_combine_df' is not defined

In [44]:
full_combine_df

NameError: name 'full_combine_df' is not defined

Lets simply build a full NBA Season database over last however many years.
Probably indexed on player ID, season, with years experience in here somewhere. 



In [ ]:
x = get_nba_data(endpt="drafthistory",params={'LeagueID':'00'})
x[x.SEASON=='2016'].head()

Using schools and draft data can save a lot of requests perhaps. 

In [ ]:
x = get_nba_data(endpt="leaguedashlineups",params={'LeagueID':'00'},
                return_url=True)
x

First approach to a comprehensive nba database will follow stats used in the clustering analysis I am to follow.

We're going to follow of this medium post which was referenced on HW3.
https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824

My man used selium to scrape from bbal reference which is probably smarter than what I did. I turn to stats.nba as the API is pretty usable.

He uses per 100 possesion, advanced and shooting data. I shall use the same but add in some defensive matchup data. 

He used career which I am going to not do. I'll use last 3 years or something. Perhaps limited by the player tracking. I also will add in some team data as what kind of offense you run and things likely has a big impact on what position you'll be playing? Maybe I shouldn't do this actually. Will consider.

Lets start building our giant beautiful database and then define a bunch of variables that can be used to selectively choose columns.

We'll use regular seson data but a playoff data analysis might be more fun later on for somehing else idk.


Since we're going to be doing a somewhat decent amount of figuring out parameters through looking at what exists on the nba site I'm going to define a little function here.

In [ ]:
url = 'http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
    
def get_params(url):
    endpt = url.split('stats/')[1].split('?')[0]
    params = {}
    paramstring = url.split('stats/')[1].split('?')[1]

    for substring in paramstring.split("&"):
        variable = substring.split('=')[0]
        value = substring.split('=')[1]
        params.update({variable:value})
    return([endpt,params])
endpt,params = get_params(url)
params.update({'SeasonType':'Regular Season'})
print(params)
advanced_2017_18 = get_nba_data(endpt=endpt,params=params)

In [ ]:
advanced_2017_18['SEASON'] = 2018
advanced_2017_18['SEASON_TYPE']='Regular Season'

keep = ['PLAYER_ID','SEASON','SEASON_TYPE','PLAYER_NAME','TEAM_ID','TEAM_ABBREVIATION','AGE','GP','MIN','OFF_RATING','DEF_RATING',
    'AST_PCT','AST_TO','AST_RATIO','OREB_PCT','DREB_PCT','REB_PCT','TM_TOV_PCT','EFG_PCT',
    'TS_PCT','USG_PCT','PACE']
advanced_2017_18 = advanced_2017_18[keep].set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])


# .loc[:,~advanced_2017_18.columns.str.contains('RANK')]\
#                                     .drop(['CFID','CFPARAMS','W_PCT','W','L','PIE',
#                                            'FGM','FGA','FGM_PG','FGA_PG'                                          
#                                           ],axis=1)\


Back to 2013-14 begninning of  player tracking data. 
Playtype might be important and that only goes so far as 2015. 

This isn't especially important because we just need some sort of clustering.


In [35]:
advanced_full = pd.DataFrame(columns=advanced_2017_18.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    
    merged['SEASON']= 2001+i
    
    merged = merged[keep]

    advanced_full = pd.concat([advanced_full,merged])

newcols = advanced_full.columns[advanced_full.columns=='PLAYER_NAME'].append(
advanced_full.columns[advanced_full.columns!='PLAYER_NAME'])
advanced_full['SEASON']=pd.Series(advanced_full.SEASON,dtype='int64')

advanced_full['PLAYER_ID']=pd.Series(advanced_full.PLAYER_ID,dtype='int64')
advanced_full['PLAYER_ID']=pd.Series(advanced_full.PLAYER_ID,dtype='str')

advanced_full = advanced_full[newcols]\
                        .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])

NameError: name 'advanced_2017_18' is not defined

In [ ]:
advanced_full.sort_index().head()

In [ ]:
endpt,params = get_params('http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Per100Possessions&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=')
params.update({'SeasonType':'Regular Season'})
per100poss = get_nba_data(endpt,params)
per100poss['SEASON'] = 2018
per100poss['SEASON_TYPE']='Regular Season'
per100poss = per100poss.loc[:,~per100poss.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','NBA_FANTASY_PTS'
                                           ,'DD2','TD3','W','L','W_PCT','MIN']
                                          ,axis=1)\
                                    .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
per100poss.columns

In [ ]:
per_100p_full= pd.DataFrame(columns=per100poss.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    merged = merged.loc[:,~merged.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','NBA_FANTASY_PTS'
                                           ,'DD2','TD3','W','L','W_PCT','MIN']
                              ,axis=1)
    merged['SEASON']= 2001+i

    per_100p_full = pd.concat([per_100p_full,merged])

newcols = per_100p_full.columns[per_100p_full.columns=='PLAYER_NAME'].append(
per_100p_full.columns[per_100p_full.columns!='PLAYER_NAME'])
per_100p_full['SEASON']=pd.Series(per_100p_full.SEASON,dtype='int64')

per_100p_full['PLAYER_ID']=pd.Series(per_100p_full.PLAYER_ID,dtype='int64')
per_100p_full['PLAYER_ID']=pd.Series(per_100p_full.PLAYER_ID,dtype='str')

per_100p_full = per_100p_full[newcols]\
                        .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
per_100p_full.head()

In [ ]:
endpt,params = get_params('http://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Scoring&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight=')
params.update({'SeasonType':'Regular Season'})
scoring = get_nba_data(endpt,params)
scoring['SEASON'] = 2018
scoring['SEASON_TYPE']='Regular Season'
scoring = scoring.loc[:,~scoring.columns.str.contains('RANK')]\
                                    .drop(['CFID','CFPARAMS','W','L','W_PCT']
                                          ,axis=1)\
                                    .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
scoring.columns

In [ ]:
scoring_full= pd.DataFrame(columns=scoring.columns)

for i in range(13,18):
    params.update({'SeasonType':'Regular Season'})
    
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = get_nba_data(endpt=endpt,params=params)
    reg_season['SEASON_TYPE'] = 'Regular Season'
    params.update({'SeasonType':'Playoffs'})
    
    playoff = get_nba_data(endpt=endpt,params=params)
    playoff['SEASON_TYPE'] = 'Playoffs'
    merged = pd.concat([reg_season,playoff])
    merged = merged.loc[:,~merged.columns.str.contains('RANK')]\
                        .drop(['CFID','CFPARAMS']
                              ,axis=1)
    merged['SEASON']= 2001+i

    scoring_full = pd.concat([scoring_full,merged])

newcols = scoring_full.columns[scoring_full.columns=='PLAYER_NAME'].append(
scoring_full.columns[scoring_full.columns!='PLAYER_NAME'])
scoring_full['SEASON']=pd.Series(scoring_full.SEASON,dtype='int64')

scoring_full['PLAYER_ID']=pd.Series(scoring_full.PLAYER_ID,dtype='int64')
scoring_full['PLAYER_ID']=pd.Series(scoring_full.PLAYER_ID,dtype='str')

scoring_full = scoring_full[newcols]\
                        .set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])
scoring_full.head()

In [ ]:
year="2017-18"
endpt = 'playerdashptshotdefend'
params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
defensive = get_nba_data(endpt=endpt,params=params)
defensive['PLAYER_ID'] = defensive.CLOSE_DEF_PERSON_ID
defensive = defensive.set_index('PLAYER_ID').drop(['CLOSE_DEF_PERSON_ID'],axis=1)
defensive.groupby('DEFENSE_CATEGORY')[['PCT_PLUSMINUS','FREQ']].describe()

In [ ]:
def defensiveUnstack(season,season_type,measures):
    endpt = 'playerdashptshotdefend'
    params = {'DateFrom':'','DateTo':'','GameSegment':'',
          'LastNGames':'0','LeagueID':'00','Location':'',
          'Month':'0','OpponentTeamID':'0','Outcome':'',
          'Period':'0','PlayerID':'0','Season':year,
          'SeasonSemgent':'','SeasonType':'Regular Season',
          'TeamID':'0','VsConference':'','VsDivision':'',
          'PerMode':'Totals','SeasonSegment':''
         }
    params.update({'Season':year,
                   'SeasonType':season_type                  
                  })
    defensive = get_nba_data(endpt=endpt,params=params)
    defensive['PLAYER_ID']=defensive.CLOSE_DEF_PERSON_ID
    defensive['SEASON'] = int(season[:4])+1
    defensive['SEASON_TYPE'] = season_type
    defensive = defensive.set_index('CLOSE_DEF_PERSON_ID')

    categories = defensive.DEFENSE_CATEGORY.unique()[1:]
    
    defensive_seperate = [defensive[defensive.DEFENSE_CATEGORY==cat][measures] for cat in categories]
    suffix_list = ['_3Pt',"_2Pt","_Lt6","_Lt10","_Gt15"]
    col_pair = defensive_seperate[0].columns
    cols = ["DEF_"+col_pair + suffix for suffix in suffix_list]
    
    for i,pair in enumerate(cols):
        defensive_seperate[i].columns=pair

    return(pd.concat(defensive_seperate,axis=1))

defensiveUnstack('2016-17','Playoffs',['PCT_PLUSMINUS','FREQ']).head()

In [ ]:
defensive = defensiveUnstack('2017-18','Playoffs',['PCT_PLUSMINUS','FREQ']).head()
defensive_full= pd.DataFrame(columns=defensive.columns)

for i in range(13,18):
    year="20"+format(i,'02d')+"-"+format(i+1,'02d')
    params.update({'Season':year})
    reg_season = defensiveUnstack(year,'Regular Season',['PCT_PLUSMINUS','FREQ'])
    reg_season['SEASON_TYPE'] = "Regular Season"
    playoff = defensiveUnstack(year,'Playoffs',['PCT_PLUSMINUS','FREQ'])
    playoff['SEASON_TYPE'] = "Playoffs"
    
    merged = pd.concat([reg_season,playoff])
    merged['PLAYER_ID']=merged.index
    merged['SEASON'] = 2001+i
    
    defensive_full = pd.concat([defensive_full,merged])


defensive_full['SEASON']=pd.Series(defensive_full.SEASON,dtype='int64')

defensive_full['PLAYER_ID']=pd.Series(defensive_full.PLAYER_ID,dtype='int64')
defensive_full['PLAYER_ID']=pd.Series(defensive_full.PLAYER_ID,dtype='str')

defensive_full = defensive_full.set_index(['PLAYER_ID','SEASON','SEASON_TYPE'])

defensive_full.sort_index().head(15)

In [ ]:
x = pd.merge(advanced_full,per_100p_full,left_index=True,right_index=True)
sum(x['MIN_x']!=x['MIN_y'])
#x.columns